<a href="https://colab.research.google.com/github/AnkitaChakraborty1/Capstone_Adversarial_Robustness/blob/main/BenchmarkedRecipes_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow and TextAttack

The following is code for training a text classification model using TensorFlow(and on top of it, the Keras API). This comes from the Tensorflow documentation ([see here](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub)).

This cell loads the IMDB dataset (using `tensorflow_datasets`, not `datasets`), initializes a simple classifier, and trains it using Keras.

In [ ]:
!pip3 install textattack[tensorflow] transformers folium==0.2.1

     |████████████████████████████████| 373 kB 5.2 MB/s 
     |████████████████████████████████| 4.0 MB 3.8 MB/s 
     |████████████████████████████████| 69 kB 7.1 MB/s 
     |████████████████████████████████| 880 kB 17.7 MB/s 
     |████████████████████████████████| 77 kB 3.2 MB/s 
     |████████████████████████████████| 596 kB 39.9 MB/s 
     |████████████████████████████████| 6.6 MB 26.6 MB/s 
     |████████████████████████████████| 69 kB 5.9 MB/s 
     |████████████████████████████████| 101 kB 3.5 MB/s 
     |████████████████████████████████| 402 kB 42.2 MB/s 
     |████████████████████████████████| 769 kB 50.6 MB/s 
     |████████████████████████████████| 325 kB 60.7 MB/s 
     |████████████████████████████████| 60 kB 7.1 MB/s 
     |████████████████████████████████| 4.9 MB 38.7 MB/s 
     |████████████████████████████████| 462 kB 59.2 MB/s 
     |████████████████████████████████| 125 kB 58.9 MB/s 
     |████████████████████████████████| 454.3 MB 16 kB/s 
     |███████████████████

In [ ]:
pip install --upgrade numpy

     |████████████████████████████████| 15.7 MB 5.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.5.0 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], 
                                  batch_size=-1, as_supervised=True)

train_examples, train_labels = tfds.as_numpy(train_data)
test_examples, test_labels = tfds.as_numpy(test_data)

model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

Version:  2.5.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete142C3H/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete142C3H/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete142C3H/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
30/30 [==============================] - 3s 72ms/step - loss: 2.3281 - accuracy: 0.4998 - val_loss: 1.1420 - val_accuracy: 0.4990
Epoch 2/40
30/30 [==============================] - 2s 64ms/step - loss: 0.7519 - accuracy: 0.5

### Attacking

For each input, our classifier outputs a single number that indicates how positive or negative the model finds the input. For binary classification, TextAttack expects two numbers for each input (a score for each class, positive and negative). We have to post-process each output to fit this TextAttack format. To add this post-processing we need to implement a custom model wrapper class (instead of using the built-in `textattack.models.wrappers.TensorFlowModelWrapper`).

Each `ModelWrapper` must implement a single method, `__call__`, which takes a list of strings and returns a `List`, `np.ndarray`, or `torch.Tensor` of predictions.

In [ ]:
import numpy as np
import torch

from textattack.models.wrappers import ModelWrapper

class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1-logits, logits), dim=1)
        return final_preds


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 45.2MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpbsavmxw7.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Let's test our model wrapper out to make sure it can use our model to return predictions in the correct format.

In [ ]:
CustomTensorFlowModelWrapper(model)(['I hate you so much', 'I love you'])

tensor([[0.2780, 0.7220],
        [0.0138, 0.9862]])

Looks good! Now we can initialize our model wrapper with the model we trained and pass it to an instance of `textattack.attack.Attack`. 

We'll use the `PWWSRen2019` recipe as our attack, and attack 10 samples.

In [ ]:
model_wrapper = CustomTensorFlowModelWrapper(model)

from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker

dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)
attack = PWWSRen2019.build(model_wrapper)

attacker = Attacker(attack, dataset)
attacker.attack_dataset()

Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9198dbc50858df8bdb0d5f18ccaf33125800af96ad8434bc8b829918c987ee8a. Subsequent calls will reuse this data.


textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  50%|█████     | 5/10 [00:00<00:00, 41.02it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (95%)]]

consistently clever and [[suspenseful]] .

consistently clever and [[cliff-hanging]] .


--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (63%)]]

it's like a " big [[chill]] " reunion of the baader-meinhof gang , only thes

[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:00<00:00, 19.65it/s]

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (70%)]]

fresnadillo has something serious to say about the ways in which [[extravagant]] [[chance]] can distort our perspective and throw us off the path of [[good]] sense .

fresnadillo has something serious to say about the ways in which [[exuberant]] [[encounter]] can distort our perspective and throw us off the path of [[dear]] sense .


--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (59%)]]

[[throws]] in enough clever and unexpected [[twists]] to make the formula feel fresh .

[[drop]] in enough clever and unexpected [[device]] to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (96%)]]

weighty and ponderous but every [[bit]] as filling as t

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 19.73it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (76%)]] --> [[Negative (59%)]]

a [[real]] audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a [[material]] audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (93%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy 

In [ ]:
from textattack import AttackArgs, Attacker
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import ModelWrapper
from textattack import Attack
from textattack.constraints.pre_transformation import (
        InputColumnModification,
        MaxModificationRate,
        RepeatModification,
        StopwordModification,
        MaxWordIndexModification
                )
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.goal_functions import UntargetedClassification,InputReduction
from textattack.search_methods import GreedyWordSwapWIR,AlzantotGeneticAlgorithm
from textattack.transformations import WordSwapQWERTY
from textattack.transformations import (
    CompositeTransformation,
    WordSwapNeighboringCharacterSwap,
    WordSwapQWERTY,
    WordSwapWordNet,
    WordSwapRandomCharacterDeletion,
    WordSwapRandomCharacterInsertion,
    WordSwapRandomCharacterSubstitution,
    WordSwapHomoglyphSwap,
    WordSwapEmbedding,
    WordDeletion,
)
from textattack.constraints.overlap import LevenshteinEditDistance
#from textattack.attack_recipe import AttackRecipe
from textattack import Attack
from textattack.constraints.overlap import MaxWordsPerturbed
from textattack.constraints.pre_transformation import (
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapMaskedLM
from abc import ABC, abstractmethod
import numpy as np

In [ ]:
class Recipe1(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
      
        transformation = WordSwapHomoglyphSwap()
        constraints = [RepeatModification(), StopwordModification()]
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe1.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()


textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:  60%|██████    | 6/10 [00:00<00:00, 31.62it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHomoglyphSwap
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (70%)]]

[[consistently]] clever and suspenseful .

[[consistentlу]] clever and suspenseful .


--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (71%)]]

it's like a " big [[chill]] " [[reunion]] of the baader-meinhof gang , only these [

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 27.83it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (72%)]]

throws in enough clever and [[unexpected]] [[twists]] to make the formula feel [[fresh]] .

throws in enough clever and [[unexpecteԁ]] [[twistѕ]] to make the formula feel [[fresհ]] .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (59%)]]

weighty and ponderous but every [[bit]] as filling as the [[treat]] of the title .

weighty and ponderous but every [[bi𝚝]] as filling as the [[trea𝚝]] of the title .


--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (76%)]] --> [[Negative (57%)]]

a real audience-pleaser that will strike a [[chord]] with anyone who's ever [[waited]] in a doctor's office , emergency room , hospital bed or insurance company office .

a real audience-pleaser that will strike a

In [ ]:
class Recipe2(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
      
        transformation = WordSwapHomoglyphSwap()
        constraints = [RepeatModification(), StopwordModification()]
        constraints.append(LevenshteinEditDistance(21))
        use_constraint = UniversalSentenceEncoder(
            threshold=0.6,
            metric="cosine",
            compare_against_original=True,
            window_size=None,
        )
        constraints.append(use_constraint)
        goal_function = UntargetedClassification(model_wrapper)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe2.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:00, 141.42it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapHomoglyphSwap
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  21
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.6
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (2): RepeatModification
    (3): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .




[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:  60%|██████    | 6/10 [00:27<00:18,  4.55s/it]

--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (70%)]]

[[consistently]] clever and suspenseful .

[[consistentlу]] clever and suspenseful .


--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (71%)]]

it's like a " big [[chill]] " [[reunion]] of the baader-meinhof gang , only these [[guys]] are more harmless pranksters than political activists .

it's like a " big [[chilⅼ]] " [[reuniоn]] of the baader-meinhof gang , only these [[guyѕ]] are more harmless pranksters than political activists .


--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (74%)]] --> [[[SKIPPED]]]

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .


--------------------------------------------- Result 

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:27<00:00,  2.75s/it]

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (59%)]]

weighty and ponderous but every [[bit]] as filling as the [[treat]] of the title .

weighty and ponderous but every [[bi𝚝]] as filling as the [[trea𝚝]] of the title .


--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (76%)]] --> [[Negative (57%)]]

a real audience-pleaser that will strike a [[chord]] with anyone who's ever [[waited]] in a doctor's office , emergency room , hospital bed or insurance company office .

a real audience-pleaser that will strike a [[chorԁ]] with anyone who's ever [[ԝaited]] in a doctor's office , emergency room , hospital bed or insurance company office .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (93%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters 

In [ ]:
class Recipe3(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
      
        transformation = WordDeletion()
        constraints = [RepeatModification(), StopwordModification()]
        constraints.append(LevenshteinEditDistance(20))
        goal_function = InputReduction(model_wrapper, maximizable=True)
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)
attack = Recipe3.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.input_reduction.InputReduction'>.
[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 54.26it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  InputReduction(
    (maximizable):  True
  )
  (transformation):  WordDeletion
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  20
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Positive (94%)]]

consistently [[clever]] and [[suspenseful]] .

consistently and .


--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (99%)]

In [ ]:
class Recipe4_bench(Attack,ABC):
    @staticmethod
    def build(model):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe4_bench.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:00<00:00, 45.29it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapHomoglyphSwap
    (3): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (4): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (5): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (6): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 45.57it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (76%)]] --> [[Negative (57%)]]

a real audience-pleaser that will strike a [[chord]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a real audience-pleaser that will strike a [[rope]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (93%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy under

In [ ]:
class Recipe5_bench(Attack,ABC):
    @staticmethod
    def build(model):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR()
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe5_bench.build(model_wrapper)
attacker = Attacker(attack, dataset)
attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  50%|█████     | 5/10 [00:00<00:00, 58.08it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapHomoglyphSwap
    (3): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (4): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (5): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 26.40it/s]

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (57%)]]

fresnadillo has something [[serious]] to say about the [[ways]] in which [[extravagant]] chance can [[distort]] our perspective and throw us off the path of good sense .

fresnadillo has something [[grave]] to say about the [[mode]] in which [[extr]] [[avagant]] chance can [[dis]] [[tort]] our perspective and throw us off the path of good sense .


--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (62%)]]

[[throws]] in enough clever and [[unexpected]] twists to make the formula [[feel]] fresh .

[[drop]] in enough clever and [[unex]] [[pected]] twists to make the formula [[finger]] fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (96%)]]

weighty and ponderous but

In [ ]:
class Recipe6_Bench(Attack,ABC):
    @staticmethod
    def build(model):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe6_Bench.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:00<00:00, 45.60it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapWordNet
    (1): WordSwapHomoglyphSwap
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (4): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (5): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 -

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 46.86it/s]

--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (76%)]] --> [[Negative (57%)]]

a real audience-pleaser that will strike a [[chord]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .

a real audience-pleaser that will strike a [[rope]] with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .


--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (93%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy under

In [ ]:
class Recipe7_bench(Attack,ABC):
    @staticmethod
    def build(model):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapWordNet(),

                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                    # (1) Insert: Insert a space into the word.
                    # Generally, words are segmented by spaces in English. Therefore,
                    # we can deceive classifiers by inserting spaces into words.
                    WordSwapRandomCharacterInsertion(
                        random_one=True,
                        letters_to_insert=" ",
                        skip_first_char=True,
                        skip_last_char=True,
                    ),
                    # (2) Delete: Delete a random character of the word except for the first
                    # and the last character
                    WordSwapRandomCharacterDeletion(
                        random_one=True, skip_first_char=True, skip_last_char=True
                    ),
                    # (5) Substitute-W
                    # (Sub-W): Replace a word with its topk nearest neighbors in a
                    # context-aware word vector space. Specifically, we use the pre-trained
                    # GloVe model [30] provided by Stanford for word embedding and set
                    # topk = 5 in the experiment.
                    WordSwapEmbedding(max_candidates=5),
                ])

        constraints = [RepeatModification(), StopwordModification()]
        goal_function = UntargetedClassification(model)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe7_bench.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:  90%|█████████ | 9/10 [00:00<00:00, 45.80it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapWordNet
    (1): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (4): WordSwapEmbedding(
        (max_candidates):  5
        (embedding):  WordEmbedding
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 --------------------------------

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 47.56it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (93%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 33.37% |
| Average num. words per input: | 15.4   |
| Avg num queries:              | 46.33  |
+-------------------------------+--------+


In [ ]:
class Recipe8_bench(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapHomoglyphSwap(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR(wir_method="delete")
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe8_bench.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset,attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.

  0%|          | 0/10 [00:00<?, ?it/s]
[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:00, 102.36it/s]
[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:  20%|██        | 2/10 [00:00<00:00, 54.71it/s] 
[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:00<00:00, 33.83it/s]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 2 / 4:  40%|████      | 4/10 [00:00<00:00, 39.06it/s]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 2 / 4:  50%|█████     | 5/10 [00:00<00:00, 42.89it/s]
[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  50%|█████     | 5/10 [00:00<00:00, 39.79it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapHomoglyphSwap
    (3): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (70%)]]

[[consistently]] clever and suspenseful .

[[consistentlу]] clever and suspenseful .


--------------------------------------------- Result 3 ----


[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:  60%|██████    | 6/10 [00:00<00:00, 26.92it/s]
[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:  70%|███████   | 7/10 [00:00<00:00, 25.21it/s]
[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:00<00:00, 23.41it/s]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:  90%|█████████ | 9/10 [00:00<00:00, 22.18it/s]

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (88%)]]

fresnadillo has something serious to say about the [[ways]] in which [[extravagant]] [[chance]] can distort our perspective and throw us off the [[path]] of good sense .

fresnadillo has something serious to say about the [[manner]] in which [[exrravagant]] [[encounter]] can distort our perspective and throw us off the [[track]] of good sense .


--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (53%)]]

throws in enough clever and [[unexpected]] [[twists]] to make the formula feel fresh .

throws in enough clever and [[unexpecetd]] [[construction]] to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (96%)]]

weighty and ponderous but every [[


[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9: 100%|██████████| 10/10 [00:00<00:00, 23.00it/s]
[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 21.90it/s]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (93%)]] --> [[[SKIPPED]]]

generates an enormous feeling of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 14.68% |
| Average num. words per input: | 15.4   |
| Avg num queries:              | 57.67  |
+-------------------------------+--------+


In [ ]:
class Recipe9_bench(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    WordSwapQWERTY(),
                    WordSwapWordNet(),
                    WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,),
                ])

        constraints = [RepeatModification(), StopwordModification()]

        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR()
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe9_bench.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/10 [00:00<?, ?it/s]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:00, 93.72it/s]

[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:  20%|██        | 2/10 [00:00<00:00, 52.52it/s]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:00<00:00, 32.65it/s]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  40%|████      | 4/10 [00:00<00:00, 36.68it/s]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 2 / 4:  40%|████      | 4/10 [00:00<00:00, 32.84it/s]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  50%|█████     | 5/10 [00:00<00:00, 35.07it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapQWERTY
    (1): WordSwapWordNet
    (2): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    )
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (70%)]] --> [[[SKIPPED]]]

lovingly photographed in the manner of a golden book sprung to life , stuart little 2 manages sweetness largely without stickiness .


--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (70%)]]

[[consistently]] clever and suspenseful .

[[consitsently]] clever and suspenseful .


--------------------------------------------- Result 3 --------------------------------------



[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:  60%|██████    | 6/10 [00:00<00:00, 26.24it/s]

[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:  70%|███████   | 7/10 [00:00<00:00, 24.83it/s]

[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:  80%|████████  | 8/10 [00:00<00:00, 23.83it/s]

[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:00<00:00, 22.67it/s]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:  90%|█████████ | 9/10 [00:00<00:00, 22.21it/s]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:00<00:00, 23.81it/s]

--------------------------------------------- Result 6 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (88%)]]

fresnadillo has something serious to say about the [[ways]] in which [[extravagant]] [[chance]] can distort our perspective and throw us off the [[path]] of good sense .

fresnadillo has something serious to say about the [[manner]] in which [[exrravagant]] [[encounter]] can distort our perspective and throw us off the [[track]] of good sense .


--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (53%)]]

throws in enough clever and [[unexpected]] [[twists]] to make the formula feel fresh .

throws in enough clever and [[unexpecetd]] [[construction]] to make the formula feel fresh .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (96%)]]

weighty and ponderous but every [[

In [ ]:
class Recipe10_bench(Attack,ABC):
    @staticmethod
    def build(model_wrapper):
        #transformation =  WordSwapNeighboringCharacterSwap(random_one=True,skip_first_char=True,skip_last_char=True,)
        transformation = CompositeTransformation(
                [
                    # (1) Swap: Swap two adjacent letters in the word.
                    WordSwapNeighboringCharacterSwap(),
                    # (2) Substitution: Substitute a letter in the word with a random letter.
                    WordSwapRandomCharacterSubstitution(),
                    # (3) Deletion: Delete a random letter from the word.
                    WordSwapRandomCharacterDeletion(),
                    # (4) Insertion: Insert a random letter in the word.
                    WordSwapRandomCharacterInsertion(),
                ])

        constraints = [RepeatModification(), StopwordModification()]
        constraints.append(LevenshteinEditDistance(21))
        constraints.append(MaxWordsPerturbed(max_percent=0.55))
        use_constraint = UniversalSentenceEncoder(
            threshold=0.6,
            metric="cosine",
            compare_against_original=True,
            window_size=None,
        )
        constraints.append(use_constraint)
        goal_function = UntargetedClassification(model_wrapper)
        # search over words based on a combination of their saliency score, and how efficient the WordSwap transform is
        search_method = GreedyWordSwapWIR()
        return Attack(goal_function, constraints, transformation, search_method)

attack = Recipe10_bench.build(model_wrapper)
attack_args = AttackArgs(num_examples=10,)
attacker = Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()


textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


  0%|          | 0/10 [00:00<?, ?it/s]

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:00, 135.78it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  21
        (compare_against_original):  True
      )
    (1): MaxWordsPerturbed(
        (max_percent):  0.55
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.6
        (window_size):  inf
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
    (3): RepeatModification
    (4): StopwordModification



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  20%|██        | 2/10 [00:05<00:22,  2.86s/it] 

[Succeeded / Failed / Skipped / Total] 1 / 0 / 1 / 2:  20%|██        | 2/10 [00:05<00:22,  2.86s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 1 / 3:  30%|███       | 3/10 [00:05<00:13,  1.92s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 2 / 4:  40%|████      | 4/10 [00:05<00:08,  1.45s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  50%|█████     | 5/10 [00:05<00:05,  1.16s/it]

[Succeeded / Failed / Skipped / Total] 2 / 0 / 3 / 5:  60%|██████    | 6/10 [00:05<00:03,  1.02it/s]

[Succeeded / Failed / Skipped / Total] 3 / 0 / 3 / 6:  60%|██████    | 6/10 [00:05<00:03,  1.02it/s]

--------------------------------------------- Result 2 ---------------------------------------------
[[Positive (87%)]] --> [[Negative (70%)]]

[[consistently]] clever and suspenseful .

[[conscstently]] clever and suspenseful .


--------------------------------------------- Result 3 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (71%)]]

it's like a " big [[chill]] " [[reunion]] of the baader-meinhof gang , only these [[guys]] are more harmless pranksters than political activists .

it's like a " big [[chiAll]] " [[reungion]] of the baader-meinhof gang , only these [[guy]] are more harmless pranksters than political activists .


--------------------------------------------- Result 4 ---------------------------------------------
[[Negative (74%)]] --> [[[SKIPPED]]]

the story gives ample opportunity for large-scale action and suspense , which director shekhar kapur supplies with tremendous skill .


--------------------------------------------- Result



[Succeeded / Failed / Skipped / Total] 4 / 0 / 3 / 7:  70%|███████   | 7/10 [00:05<00:02,  1.18it/s]

[Succeeded / Failed / Skipped / Total] 5 / 0 / 3 / 8:  80%|████████  | 8/10 [00:05<00:01,  1.34it/s]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9:  90%|█████████ | 9/10 [00:06<00:00,  1.50it/s]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 3 / 9: 100%|██████████| 10/10 [00:06<00:00,  1.66it/s]

[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:06<00:00,  1.65it/s]

--------------------------------------------- Result 7 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (72%)]]

throws in enough clever and [[unexpected]] [[twists]] to make the formula feel [[fresh]] .

throws in enough clever and [[ulexpected]] [[twiists]] to make the formula feel [[fesh]] .


--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (99%)]] --> [[Negative (59%)]]

weighty and ponderous but every [[bit]] as filling as the [[treat]] of the title .

weighty and ponderous but every [[bVit]] as filling as the [[Zreat]] of the title .


--------------------------------------------- Result 9 ---------------------------------------------
[[Positive (76%)]] --> [[Negative (57%)]]

a real audience-pleaser that will strike a [[chord]] with anyone who's ever [[waited]] in a doctor's office , emergency room , hospital bed or insurance company office .

a real audience-pleaser that will strike 